<a href="https://colab.research.google.com/github/tanmayispiddy07/LOcus_wed/blob/main/phase_generator_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
import os
if 'COLAB_RELEASE_TAG' in os.environ:
  from google.colab import userdata
  import pathlib
  pathlib.Path('/content/client_secret.json').write_text(userdata.get('CLIENT_SECRET'))

  # Use `--no-browser` in colab
  !gcloud auth application-default login --no-browser --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'
else:
  !gcloud auth application-default login --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'

You are authorizing client libraries without access to a web browser. Please run the following command on a machine with a web browser and copy its output back here. Make sure the installed gcloud version is 372.0.0 or newer.

gcloud auth application-default login --remote-bootstrap="https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=9418359443-nldovmhh4iq7db59e4311n94tounuk1g.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgenerative-language.tuning&state=P12tN66K2KgIZOyWvdmLhuqHWX6TZt&access_type=offline&code_challenge=8JXe5o0KVwDaCdd9oFHo1IcRNp4Fr9MGaWr1tvqLJPM&code_challenge_method=S256&token_usage=remote"


Enter the output of the above command: https://localhost:8085/?state=P12tN66K2KgIZOyWvdmLhuqHWX6TZt&code=4/0AQlEd8zLP1zANRTDjCeGWp9bVDykZa-zWtItJciGnXB9KMOE72k6182DdjnoUi1YBKh_lg&scope=https://www.googleapis.com/auth/cloud-platform%20https://www.googleapis.com/auth/generative

In [ ]:
!pip install -q google-generativeai

In [3]:
import google.generativeai as genai

In [4]:
for i, m in zip(range(5), genai.list_tuned_models()):
  print(m.name)

tunedModels/phasegenerator2-8kybdxroccyf
tunedModels/phrasegenerator3-ktb3b1xs5lcb
tunedModels/phasegenerator6
tunedModels/phase-generator-7
tunedModels/phase-generator-8


In [5]:
base_model = [
    m for m in genai.list_models()
    if "createTunedModel" in m.supported_generation_methods][1]
base_model

Model(name='models/gemini-1.5-flash-001-tuning',
      base_model_id='',
      version='001',
      display_name='Gemini 1.5 Flash 001 Tuning',
      description='Fast and versatile multimodal model for scaling across diverse tasks',
      input_token_limit=16384,
      output_token_limit=8192,
      supported_generation_methods=['generateContent', 'countTokens', 'createTunedModel'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=64)

In [ ]:
import json

# Open the JSON file for reading
with open('/content/train.json', 'r') as f:
    # Try to load the JSON data, handling potential errors
    try:
        data = json.load(f)
    except json.JSONDecodeError as e:
        # Print a more informative error message if decoding fails
        print(f"Error decoding JSON: {e}")
        # Optionally, print the problematic line from the JSON file
        f.seek(0)
        for i, line in enumerate(f):
            if i == e.lineno - 1:
                print(f"Problematic line ({e.lineno}): {line.strip()}")
                break
        exit()  # Exit the script if there's a JSON decoding error


In [ ]:
df = pd.read_json('/content/train.json')

In [ ]:
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
df_sampled = pd.DataFrame()
df_sampled['text_input']= df_shuffled['Tweet'].head(120)
df_sampled['output'] = df_shuffled['Target'].head(120)


In [ ]:
name = f'phase-generator-8'
operation = genai.create_tuned_model(

    source_model=base_model.name,
    training_data=df_sampled,
    id = name,
    epoch_count = 2,
    batch_size=4,
    learning_rate=0.0001,
)

In [ ]:
import time

for status in operation.wait_bar():
  time.sleep(30)

  0%|          | 0/150 [00:00<?, ?it/s]

In [ ]:
model = genai.get_tuned_model(f'tunedModels/phase-generator-7')

model

TunedModel(name='tunedModels/phase-generator-7',
           source_model='models/gemini-1.5-flash-001-tuning',
           base_model='models/gemini-1.5-flash-001-tuning',
           display_name='',
           description='',
           temperature=1.0,
           top_p=0.95,
           top_k=64,
           state=<State.ACTIVE: 2>,
           create_time=datetime.datetime(2024, 8, 21, 10, 28, 25, 455110, tzinfo=datetime.timezone.utc),
           update_time=datetime.datetime(2024, 8, 21, 10, 44, 14, 271836, tzinfo=datetime.timezone.utc),
           tuning_task=TuningTask(start_time=datetime.datetime(2024, 8, 21, 10, 28, 26, 74889, tzinfo=datetime.timezone.utc),
                                  complete_time=datetime.datetime(2024, 8, 21, 10, 44, 14, 271836, tzinfo=datetime.timezone.utc),
                                  snapshots=[...],
                                  hyperparameters=Hyperparameters(epoch_count=5,
                                                                  ba

In [ ]:
import pandas as pd
import seaborn as sns

model = operation.result()

snapshots = pd.DataFrame(model.tuning_task.snapshots)

sns.lineplot(data=snapshots, x = 'epoch', y='mean_loss')

NameError: name 'operation' is not defined

In [10]:
import pandas as pd
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import json

In [6]:
df = pd.read_csv('/content/test_20.csv' , encoding='latin-1')

In [47]:
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",

}

model = genai.GenerativeModel(
  model_name="tunedModels/phase-generator-7",
  generation_config=generation_config,

)

chat_session = model.start_chat(
  history=[
    {
      "role": "model",
      "parts": [
        "```json\n{\n  \"target\": \"Creationism\",\n  \"stance\": \"Negative\" \n}\n``` \n",
      ],
    },
    {
      "role": "model",
      "parts": [
        "```json\n{\n  \"target\": \"MeToo\", \n  \"stance\": \"Against\" \n}\n```",
      ],
    },

    {
      "role": "model",
      "parts": [
        "```json\n{\n  \"target\": \"ClimateChange\",\n  \"stance\": \"Favor\" \n}'''",
      ],
    },
  ]
)

In [8]:
def predict_target_and_stance(tweet_text):
    json_response = chat_session.send_message(
        ["find the implicit target of this tweet"+tweet_text],safety_settings={
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        }
    )

    return json_response.to_dict()

In [12]:
response_json = predict_target_and_stance(df['Tweet'][180])
json_text=response_json['candidates'][0]['content']['parts'][0]['text']
json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
print(response_json)

{'candidates': [{'content': {'parts': [{'text': 'Death Penalty'}], 'role': 'model'}, 'finish_reason': 1, 'index': 0, 'safety_ratings': [{'category': 9, 'probability': 1, 'blocked': False}, {'category': 8, 'probability': 1, 'blocked': False}, {'category': 7, 'probability': 1, 'blocked': False}, {'category': 10, 'probability': 1, 'blocked': False}], 'token_count': 0, 'grounding_attributions': []}], 'usage_metadata': {'prompt_token_count': 130, 'candidates_token_count': 2, 'total_token_count': 132, 'cached_content_token_count': 0}}


In [13]:
df['Tweet'][190]

'the number of years on the average spent in death row is years'

{'cloning', 'cloningcloning'}

In [14]:
df['Predictions'] = None

for i in range(100):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [43]:
for i in range(100,200):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [48]:
for i in range(200,300):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [19]:
for i in range(300,400):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [20]:
for i in range(400,500):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [22]:
for i in range(500,600):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [25]:
for i in range(600,700):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [26]:
for i in range(700,800):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [27]:
for i in range(800,900):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [30]:
for i in range(900,1000):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [31]:
for i in range(1000,1100):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [33]:
for i in range(1200,1300):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [34]:
for i in range(1300,1400):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [36]:
for i in range(1400,1500):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [38]:
for i in range(1500,1600):
  response_json = predict_target_and_stance(df['Tweet'][i])
  json_text=response_json['candidates'][0]['content']['parts'][0]['text']
  json_text_cleaned = json_text.strip('```json\n').strip('\n``` ')
  df.at[i, 'Predictions'] = json_text_cleaned

In [50]:
phrases_to_remove = ['gun control']

# Remove the specified phrases from the text
for i in range(len(df)):
    p = df.loc[i, 'Predictions']
    if p is not None:  # Check if p is not None before applying replace
        for phrase in phrases_to_remove:
            p = p.replace(phrase, '').strip()
        df.at[i, 'Predictions'] = p # Update the DataFrame

In [52]:
df[30:330]

,Unnamed: 0.1,Unnamed: 0,Tweet,Target,Stance,ID,Target_ID,Predictions
320,45758,45759,whether it should be carried out or not has be...,death penalty,NONE,9,10,
321,45759,45760,no civilian s job description should include k...,death penalty,AGAINST,9,10,
322,45760,45761,granted we are not able to ask the dead whethe...,death penalty,FAVOR,9,10,death penaltydeath penalty
323,45761,45762,this system is broken,death penalty,NONE,9,10,
324,45762,45763,pro simply because an execution method may res...,death penalty,FAVOR,9,10,death penaltydeath penalty
325,45763,45764,if improvements are needed in the system of re...,death penalty,NONE,9,10,
326,45764,45765,scholars psychologists activists all pour into...,death penalty,NONE,9,10,death penaltydeath penalty
327,45765,45766,t he claim that t his sentence rests on the ir...,death penalty,NONE,9,10,
328,45766,45767,death is an experience that can not be experie...,death penalty,NONE,9,10,death penaltydeath penalty
329,45767,45768,murder always tragic seems to be a more heinou...,death penalty,NONE,9,10,gunclontrol


In [54]:

from sklearn.metrics import accuracy_score, classification_report

# Assuming you want to evaluate on the first 200 samples
target_accuracy = accuracy_score(df['Target'][:200], df['Predictions'][:200]) # Use 'Predictions' column from df
target_classification_report = classification_report(df['Target'][:200], df['Predictions'][:200])

print(f"Target Accuracy: {target_accuracy}")
print("Target Classification Report:")
print(target_classification_report)

Target Accuracy: 0.42
Target Classification Report:
                                                                                                                                precision    recall  f1-score   support

                                                                                                                                     0.00      0.00      0.00         0
Genetically Modified Organisms (GMOs)cloningminimum wagemarijuana legalizationabortionminimum wagemarijuana legalizationaborti       0.00      0.00      0.00         0
                                                                                                                      abortion       0.00      0.00      0.00         0
                                                                                                                       cloning       1.00      0.88      0.93        96
                                                                                                           

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [ ]:
import pandas as pd
import seaborn as sns

model = operation.result()

snapshots = pd.DataFrame(model.tuning_task.snapshots)

sns.lineplot(data=snapshots, x = 'epoch', y='mean_loss')